1️⃣ Secrets & API-key bootstrap

In [ ]:
# ────────────────────────────────────────────────────────────────────────
#  SECRETS → LOCAL FILES  |  runs before any other import
# ────────────────────────────────────────────────────────────────────────
import os, base64, pathlib
import openai
from openai import OpenAI

def _decode(secret_name: str, out_file: str):
    """Decode a Base-64 GitHub secret onto disk."""
    pathlib.Path(out_file).write_bytes(
        base64.b64decode(os.environ[secret_name])
    )

_decode("CLIENT_SECRETS_JSON", "client_secrets.json")
_decode("TOKEN_JSON",          "token.json")

# OpenAI key comes from the repo secret
openai.api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI()                              # <- used later for chat + TTS

print("✅ Secrets decoded, OpenAI client ready.")


2️⃣ Parameter cell

In [ ]:
# PARAMETERS  (Papermill / run-notebook overwrites these at run-time)
GAMEPLAY_URL = None                         # GitHub secret injects real link
TITLE_TEXT   = "Daily DALLE Short"          # you can override this too
OPENAI_MODEL = "gpt-4o"


🔐 3. Authenticate with YouTube

In [ ]:
# 🔐 2. Authenticate with YouTube (automatic – no browser needed)
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

SCOPES = [
    "https://www.googleapis.com/auth/youtube.upload",
    "https://www.googleapis.com/auth/youtube.force-ssl",
]

creds = Credentials.from_authorized_user_file("token.json", SCOPES)
youtube = build("youtube", "v3", credentials=creds)

print("✅ YouTube Data API authenticated via refresh-token.")


🚀 4.0 Cockpit: Your Customizable Inputs

In [ ]:
# ─── 0. Cockpit: Auto‑Generated News Topic via Reddit r/worldnews ──────────
import random, time
import praw
import requests
from bs4 import BeautifulSoup

# 1) Initialize PRAW Reddit client
reddit = praw.Reddit(
    client_id="IPF2FtGAO8sLhzTUbEGSsQ",
    client_secret="RLO_IjGT9qIjnaYPxlVctFF3XL4Cjg",
    user_agent="DailyNewsMax by /u/Rexxus25"
)

# 2) Fetch top submissions from r/worldnews in the past 48 hours
sub = reddit.subreddit("worldnews")
now = time.time()
two_days_ago = now - 48 * 3600

# Grab the top 50 from the past week, filter to last 48 h, sort by score
posts = list(sub.top(time_filter="week", limit=50))
recent = [p for p in posts if p.created_utc >= two_days_ago]
recent_sorted = sorted(recent, key=lambda p: p.score, reverse=True)
top10 = recent_sorted[:20] if recent_sorted else posts[:20]

# 3) Pick one at random
selected = random.choice(top10)
background_topic = selected.title
article_url      = selected.url

# ─── NEW: Fetch the full article text ────────────────────────────────
resp = requests.get(article_url, timeout=10)
soup = BeautifulSoup(resp.text, "html.parser")
paragraphs = soup.find_all("p")
article_text = "\n".join(p.get_text() for p in paragraphs)

print("🔄 Auto‑selected news topic:", background_topic)
print("🔗 Article URL:", article_url)
print("📄 Article snippet:", article_text[:200].replace("\n", " "), "…")

# === COCKPIT SETTINGS ===
clip_duration   = 61   # seconds (you can adjust for testing)
outro_text      = "That's it for today! Follow for more news tomorrow."
outro_fontsize  = 55
outro_position  = ("center","center")
outro_duration  = 3    # seconds

output_width, output_height = 1080, 1920
pexels_per_search = 15

print("✅ Cockpit configured:")
print(f"   • Topic         = {background_topic!r}")
print(f"   • URL           = {article_url}")
print(f"   • Clip duration = {clip_duration}s (+ outro {outro_duration}s)")
print(f"   • Resolution    = {output_width}×{output_height}")
print(f"   • Pexels search = {pexels_per_search} clips tested")


🎥 4.05 Pexels Search & Download Helpers

In [ ]:
import requests
PEXELS_API_KEY = "dF2GrslU19FhJYkOVLWNLO4Mz8Bk5UPDcmVILSCPeQDForUFqnRvX5yH"

def search_pexels_videos(query, per_page=5):
    url = "https://api.pexels.com/videos/search"
    headers = {"Authorization": PEXELS_API_KEY}
    params = {"query": query, "per_page": per_page}
    r = requests.get(url, headers=headers, params=params)
    r.raise_for_status()
    return r.json()["videos"]

def download_pexels_video(video_json, fname):
    files = video_json.get("video_files", [])
    if not files:
        raise RuntimeError("No video_files in JSON")
    # pick highest resolution
    files.sort(key=lambda f: f.get("width",0)*f.get("height",0), reverse=True)
    url = files[0]["link"]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(fname, "wb") as f:
            for chunk in r.iter_content(8192):
                if chunk:
                    f.write(chunk)
    return fname

🔧 4.1 Generate Text Segments Cell

In [ ]:
# ─── 4.1 Generate Chunked Text Segments via ChatGPT ─────────────────────
import json, re, random
from datetime import datetime

intros = [
    "Hey everyone, welcome to DailyNewsMax! Today’s story is about",
    "Hello and thanks for tuning in—DailyNewsMax here with today’s headline:",
    "Good day, folks! This is DailyNewsMax, covering today’s top story about",
    "What’s up, NewsMaxers? Today we’re looking at",
]
today_str = datetime.now().strftime("%B %d, %Y")

prompt = f"""
You are a Professional Breaking News Script Generator for DailyNewsMax. Create an urgent, high-energy dialogue between the host and the audience for a {clip_duration}-second YouTube Short video. Ensure there is enough generated text to fill out a narrated clip of {clip_duration} seconds, using:

**STORY CONTEXT**
- HEADLINE: "{background_topic}"
- SOURCE: "{article_url}"
- BODY: "{article_text}"

**FORMAT RULES**
1. NO SPEAKER LABELS
2. Use VIRAL HOOK TECHNIQUES:
   - Start with a shocking statistic or phrase.
3. STRUCTURE LIKE TOP NEWS SHORTS:
   [OPENING 0–3s] Attention-grabbing hook, beginning with one of:
     "{random.choice(intros)} {background_topic}."
   [TEASE 3–7s] Tease with urgency and date:
     "This just broke on {today_str} from {article_url.split('/')[2]}..."
   [PAYOFF 7–{clip_duration}s] Rapid-fire facts + CTA
     - Alternate perspectives (Fact → Reaction)
     - End with "Tap follow NOW for updates"

4. Include enough relevant detail from the body to inform the viewer.
5. Keep it concise—no more than 20 words per chunk, natural breakpoints.

Return ONLY a JSON array of three arrays—one per segment—like:
[
  ["Hook chunk 1", "Hook chunk 2"],
  ["Tease chunk 1", "Tease chunk 2"],
  ["Payoff chunk 1", "Payoff chunk 2", ...]
]
"""

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.9,
    max_tokens=760
)

raw = resp.choices[0].message.content.strip()
json_text = raw[raw.find('['):raw.rfind(']')+1]
json_text = re.sub(r',\s*(\])', r'\1', json_text)

try:
    texts_nested = json.loads(json_text)
except json.JSONDecodeError as e:
    raise ValueError(f"Failed to parse JSON:\n{json_text}\n\nOriginal response:\n{raw}") from e

print("✅ Generated nested text segments:", texts_nested)

🔧 4.2 Generate NARRATION for Text Segments Cell

In [ ]:
# ─── 4 .2  Flatten text & generate TTS narration ───────────────────────
# MoviePy ≥ 2.x no longer provides the `moviepy.editor` aggregator.
# Import AudioFileClip directly from the new audio sub-package:
from moviepy.audio.io.AudioFileClip import AudioFileClip

def generate_tts(text: str, filename: str) -> str:
    """Generate text-to-speech using OpenAI TTS and save to *filename*."""
    response = client.audio.speech.create(        # uses the global `client`
        model="tts-1",
        voice="echo",
        input=text
    )
    response.stream_to_file(filename)
    return filename

def calculate_precise_timings(flat_segments):
    """Back-propagate precise timings once we know exact audio durations."""
    current_start = 0.0
    for segment in flat_segments:
        segment["start"] = current_start
        segment["end"]   = current_start + segment["duration"]
        current_start    = segment["end"]
    return flat_segments

# ─── 1) Flatten the nested text into sequential segments ───────────────
segment_word_counts = [
    sum(len(chunk.split()) for chunk in seg) for seg in texts_nested
]
total_words      = sum(segment_word_counts)
seconds_per_word = clip_duration / total_words

flat_segments, t = [], 0.0
for seg in texts_nested:
    for chunk in seg:
        dur_est = len(chunk.split()) * seconds_per_word
        flat_segments.append({"text": chunk, "start": t, "duration": dur_est})
        t += dur_est

# ─── 2) Generate individual TTS files ───────────────────────────────────
narration_files = []
for idx, seg in enumerate(flat_segments):
    fn = f"narr_{idx}.mp3"
    print(f"🔈 TTS chunk {idx}: “{seg['text']}” → {fn}")
    generate_tts(seg["text"], fn)
    narration_files.append((fn, seg["start"]))

# ─── 3) Re-measure each clip’s real duration, then fix timings ──────────
for i, (fn, _) in enumerate(narration_files):
    audio_clip = AudioFileClip(fn)
    flat_segments[i]["duration"] = audio_clip.duration

flat_segments   = calculate_precise_timings(flat_segments)
narration_files = [
    (fn, seg["start"])
    for fn, seg in zip([f[0] for f in narration_files], flat_segments)
]

print("✅ Synced narration segments:", flat_segments)


5-7 Video Editing

In [ ]:
# ─── 0) ✱ NEW – fetch gameplay from URL (runs once) ──────────────────────
import os, pathlib, subprocess, shlex

dst = pathlib.Path("gameplay_trimmed.mp4")
if not dst.exists():
    file_id = os.environ["GAMEPLAY_URL"].split("id=")[-1]
    subprocess.run(shlex.split(f"gdown --id {file_id} -O {dst}"), check=True)
print("✓ gameplay at", dst.resolve())

# ─── 5–7 Video Editing + Composite Narration ─────────────────────────────
import random, textwrap
import numpy as np
from concurrent.futures import ThreadPoolExecutor

# ⬇️ NEW — universal MoviePy ≥/≤ 2.0 helpers & imports
# ── editing code block start ─────────────────────────────────────────────
from moviepy import (
    VideoFileClip, CompositeVideoClip, concatenate_videoclips,
    ColorClip, ImageClip, AudioFileClip, CompositeAudioClip,
    vfx, afx
)

# ─── Generic helper to apply an *effect* on any MoviePy version ──────────
def _apply_fx(clip, effect):
    """
    Safely apply an *effect* object on *clip* across MoviePy versions.
    Works with:
      • clip.fx(...)                – MoviePy ≤ 1.x and most 2.x builds
      • clip.with_effect(...)       – some 2.x snapshots
      • clip.with_fx(...)           – rare alt name that appeared briefly
      • clip.with_effects([effect]) – MoviePy ≥ 2.0 official
    """
    if hasattr(clip, "fx"):
        return clip.fx(effect)
    if hasattr(clip, "with_effect"):
        return clip.with_effect(effect)
    if hasattr(clip, "with_fx"):
        return clip.with_fx(effect)
    if hasattr(clip, "with_effects"):
        return clip.with_effects([effect])
    raise RuntimeError(
        "No recognised method to apply effects on this MoviePy build."
    )

# ─── 1) audio_loop (works on any version) ───────────────────────────────
def audio_loop(clip, *, duration=None, n=None):
    """Repeat *clip* until it reaches *duration* seconds or *n* loops."""
    # MoviePy ≤ 1.x still ships the old helper in afx
    if getattr(afx, "audio_loop", None):
        return afx.audio_loop(clip, duration=duration, n=n)

    # MoviePy ≥ 2.0 effect class
    try:
        from moviepy.audio.fx.AudioLoop import AudioLoop
        return _apply_fx(clip, AudioLoop(duration=duration, n=n))
    except ModuleNotFoundError:
        # Fallback: rely on the generic vfx.Loop (works on audio too)
        return _apply_fx(clip, vfx.Loop(duration=duration, n=n))

# ─── 2) resize_fx  (handles .resize ⇆ .resized rename)  ─────────────────
def resize_fx(clip, newsize):
    """Version-agnostic resize effect."""
    if hasattr(clip, "resized"):      # MoviePy ≥ 2.0
        return clip.resized(newsize)
    return clip.resize(newsize)       # MoviePy ≤ 1.x

# ─── 3) loop_fx  (video or audio) ───────────────────────────────────────
def loop_fx(clip, *, duration=None, n=None):
    """Loop *clip* via the vfx.Loop effect, regardless of version."""
    return _apply_fx(clip, vfx.Loop(duration=duration, n=n))

# ─── 4) subclip_fx  (max-compat temporal trimming) ──────────────────────
def subclip_fx(clip, start, end):
    """
    Return a slice of *clip* between *start* – *end* seconds, whatever
    MoviePy version is installed.
    """
    # A) Classic API  (MoviePy ≤ 1.x   & most early 2.x dev builds)
    if hasattr(clip, "subclip"):
        return clip.subclip(start, end)

    # B) Official 2.0+ method name
    if hasattr(clip, "time_slice"):
        return clip.time_slice(start_time=start, end_time=end)

    # C) 2.0 snapshots that expose a Trim/Subclip *effect* class instead
    try:
        from moviepy.video.fx.Subclip import Subclip as _Subclip
        return _apply_fx(clip, _Subclip(start_time=start, end_time=end))
    except ModuleNotFoundError:
        pass
    try:
        from moviepy.video.fx.trim import trim as _trim
        return _trim(clip, start_time=start, end_time=end)
    except ModuleNotFoundError:
        pass

    # D) Last-ditch fallback – we're on some exotic build that renamed the API
    if start == 0:
        # 1) most historical versions
        if hasattr(clip, "set_duration"):
            return clip.set_duration(end)
        # 2) MoviePy 2.0+ renamed setter
        if hasattr(clip, "with_duration"):
            return clip.with_duration(end)
        # 3) extremely early 2.x snapshots used `with_end`
        if hasattr(clip, "with_end"):
            return clip.with_end(end)
        # 4) give up – at least return the original clip rather than crash
        return clip
    raise RuntimeError("No compatible subclip/time-slice implementation found")

# ─── 5) crop_fx  (covers every naming convention so far) ────────────────
def crop_fx(clip, **kwargs):
    """
    Universal crop that works with MoviePy 1.x and 2.x.
    Accepts the classic keyword args (x1, y1, x2, y2, width, height…).
    """
    # (a) Native instance methods first
    if hasattr(clip, "cropped"):      # MoviePy ≥ 2.0 preferred
        return clip.cropped(**kwargs)
    if hasattr(clip, "crop"):         # MoviePy ≤ 1.x
        return clip.crop(**kwargs)

    # (b) New effect class (MoviePy ≥ 2.0 snapshots)
    try:
        from moviepy.video.fx.Crop import Crop as _Crop
        return _apply_fx(clip, _Crop(**kwargs))
    except ModuleNotFoundError:
        pass

    # (c) Very old functional helper
    try:
        from moviepy.video.fx.crop import crop as _crop_func
        return _crop_func(clip, **kwargs)
    except ModuleNotFoundError as e:
        raise RuntimeError("No compatible crop implementation found") from e

# ── editing code block end ───────────────────────────────────────────────

# ── imports needed by text-clip generator ───────────────────────────────
from PIL import Image, ImageDraw, ImageFont
from freesound import FreesoundClient
import requests

# ─── Font, canvas & divider settings ────────────────────────────────────
text_font      = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
text_fontsize  = 60
W, H           = output_width, output_height
half_h         = H // 2
divider_height = 30                     # white bar thickness

# ─── 1) Top half: gameplay with 12 % crop, resize & loop ────────────────
gameplay = VideoFileClip("gameplay_trimmed.mp4")
h_crop   = int(gameplay.h * 0.12)
gameplay = crop_fx(gameplay, y1=h_crop, y2=gameplay.h - h_crop)
gameplay = loop_fx(resize_fx(gameplay, (W, half_h)), duration=clip_duration)

# ─── 2) Bottom half: Pexels background – squash to fit ──────────────────
videos  = search_pexels_videos(background_topic, per_page=pexels_per_search)
paths   = [f"clip_{i}.mp4" for i, _ in enumerate(videos)]
with ThreadPoolExecutor() as ex:
    ex.map(lambda args: download_pexels_video(*args), zip(videos, paths))

subclips, acc = [], 0
for p in paths:
    clip    = VideoFileClip(p)
    remain  = clip_duration - acc
    if remain <= 0:
        break
    take    = min(clip.duration, remain)
    resized = resize_fx(clip, (W, half_h))
    subclip = subclip_fx(resized, 0, take)  # Removed .set_duration()

    # Version-agnostic duration setting
    if hasattr(subclip, "with_duration"):
        subclip = subclip.with_duration(take)
    elif hasattr(subclip, "set_duration"):
        subclip = subclip.set_duration(take)

    subclips.append(subclip)
    acc += take

background_bottom = concatenate_videoclips(subclips, method="compose") \
                    .set_duration(clip_duration)

# ─── 3) Divider: 30 px white bar at the seam ────────────────────────────
divider = ColorClip((W, divider_height), color=(255, 255, 255),
                    duration=clip_duration).set_position(
                    ("center", half_h - divider_height // 2))

# ─── 4) Text-clip generator  (now defined **before** first use) ─────────
def make_text_clip(txt, duration, frame_size, font_path, fontsize,
                   fill, position):
    """
    Renders *txt* to a semi-transparent boxed overlay and returns a
    MoviePy ImageClip sized to *frame_size*.
    """
    font   = ImageFont.truetype(font_path, fontsize)
    max_w  = int(frame_size[0] * 0.9)
    avg_w  = font.getbbox("A")[2]
    wrap_w = max_w // avg_w
    lines  = textwrap.wrap(txt, width=wrap_w)

    # measure multiline block
    dummy   = ImageDraw.Draw(Image.new("RGB", (1, 1)))
    bboxes  = [dummy.textbbox((0, 0), ln, font=font) for ln in lines]
    widths  = [x1 - x0 for x0, _, x1, _ in bboxes]
    heights = [y1 - y0 for _, y0, _, y1 in bboxes]
    block_w = max(widths)  + 20
    block_h = sum(heights) + 5 * (len(lines) - 1) + 20

    # transparent RGBA canvas
    img   = Image.new("RGBA", frame_size, (0, 0, 0, 0))
    draw  = ImageDraw.Draw(img)
    x0,y0 = (frame_size[0] - block_w)//2, (frame_size[1] - block_h)//2
    draw.rectangle([x0, y0, x0 + block_w, y0 + block_h], fill=(0, 0, 0, 128))

    y = y0 + 10
    for ln in lines:
        w = draw.textbbox((0, 0), ln, font=font)[2]
        draw.text((x0 + (block_w - w)//2, y), ln, font=font, fill=fill)
        y += font.getbbox(ln)[3] - font.getbbox(ln)[1] + 5

    return ImageClip(np.asarray(img)).set_duration(duration).set_position(position)

# ─── (the “Build Synced Text Clips” loop begins right after this) ───────

# (build text_clips → outro → composite → background music → export)
# ─── 5) Build Synced Text Clips ────────────────────────────────────────
text_clips = []
for idx, (fn, start) in enumerate(narration_files):
    audio = AudioFileClip(fn)
    duration = audio.duration
    tc = make_text_clip(
        flat_segments[idx]["text"],
        duration,
        (W, H),
        text_font, text_fontsize,
        "white", ("center","center")
    ).set_start(start).set_duration(duration)
    text_clips.append(tc)

# ─── 6) Outro Clip ─────────────────────────────────────────────────────
outro_txt_clip = make_text_clip(
    outro_text, outro_duration, (W, H),
    text_font, text_fontsize, "white", ("center","center")
)
outro_bg = ColorClip((W, H), (0,0,0), duration=outro_duration)
outro_clip = CompositeVideoClip([outro_bg, outro_txt_clip]).set_duration(outro_duration)

# ─── 7) Composite Split Screen ─────────────────────────────────────────
split = CompositeVideoClip(
    [gameplay.set_position(("center","top")),
     background_bottom.set_position(("center","bottom")),
     divider] + text_clips,
    size=(W, H)
)
video_final = concatenate_videoclips([split, outro_clip], method="compose")

# ─── 8) Background Music ───────────────────────────────────────────────
fs = FreesoundClient(); fs.set_token("SvkdYKjMtv5lFj5ojxbJ8MPA9dmr8okzsU9pOQIi")
def find_tracks(q, lic):
    return fs.text_search(
        query=f"{q} music", filter=f'license:"{lic}"',
        sort="rating_desc", fields="id,name,license,previews",
        per_page=10
    ).results

tracks = (
    find_tracks(background_topic, "Creative Commons 0")
    or find_tracks(background_topic, "Creative Commons Attribution")
    or find_tracks("background music", "Creative Commons 0")
)

if tracks:
    snd = random.choice(tracks)
    url = snd["previews"].get("preview-hq-mp3") or snd["previews"].get("preview_hq_mp3")
    resp = requests.get(url)
    with open("music.mp3","wb") as f:
        f.write(resp.content)
    bg_music = audio_loop(
        AudioFileClip("music.mp3").volumex(0.2),
        duration=video_final.duration
    )
else:
    bg_music = None

# ─── 9) Final Audio Mix ───────────────────────────────────────────────
narr_clips = [AudioFileClip(fn).set_start(start) for fn, start in narration_files]
audio_layers = narr_clips + ([bg_music] if bg_music else [])
final_audio = CompositeAudioClip(audio_layers)
final = video_final.set_audio(final_audio)

# ─── 10) Export ───────────────────────────────────────────────────────
final.write_videofile(
    "shorts_final.mp4",
    codec="libx264",
    audio_codec="aac",
    threads=4,
    preset="ultrafast",
    ffmpeg_params=["-crf","23"]
)


KeyError: 'GAMEPLAY_URL'

#@📝 8. Generate Metadata

In [ ]:
# ─── 8. Generate Metadata (title ≠ background_topic) ───────────────────────
def generate_metadata(topic):
    prompt = (
        f"Generate a YouTube Short title that is about '{topic}' "
        f"but does NOT exactly match it. The title should be captivating or catchy and be a generalized title of the '{topic}'.  Then provide a concise description "
        f"(under 100 words) and 5-6 relevant SEO‑friendly tags, but always #trending, #fyp and #viral #viralshorts included (comma‑separated)."
    )
    resp = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user","content":prompt}],
        temperature=0.7, max_tokens=200
    )
    text = resp.choices[0].message.content.strip()

    # Parse metadata
    title, desc, tags = None, None, []
    for line in text.splitlines():
        low = line.lower()
        if low.startswith("title:"):
            title = line.split(":",1)[1].strip()
        if low.startswith("description:"):
            desc = line.split(":",1)[1].strip()
        if low.startswith("tags:"):
            tags = [t.strip() for t in line.split(":",1)[1].split(",")]

    # Ensure Shorts requirements
    if "#shorts" not in [t.lower() for t in tags]:
        tags.append("#shorts")
    if any(len(tag) > 25 for tag in tags):
        tags = [tag[:25] for tag in tags]

    return title, desc, tags

title, description, tags = generate_metadata(background_topic)
print("📝 Metadata:\n ", title, "\n ", description, "\n ", tags)

📤 9. Upload to YouTube

In [ ]:
# ─── 9. Upload to YouTube ───────────────────────────────────────────────
from googleapiclient.http import MediaFileUpload
from googleapiclient.errors import HttpError
import time, os

def upload_video_to_youtube(
    file_path, title, description, tags,
    category_id=22, privacy="public",
    chunk_size=1024*1024, max_retries=5
):
    # Enforce Shorts requirements
    if not any(t.lower() == "#shorts" for t in tags):
        tags.append("#shorts")

    body = {
        "snippet": {
            "title": f"{title} #shorts",
            "description": f"{description}\n\n#shorts",
            "tags": tags,
            "categoryId": str(category_id)
        },
        "status": {
            "privacyStatus": privacy,
            "selfDeclaredMadeForKids": False
        },
        "contentDetails": {
            "duration": "PT60S"  # Force recognition as Short
        }
    }

    media = MediaFileUpload(
        file_path,
        mimetype='video/mp4',
        chunksize=chunk_size,
        resumable=True
    )

    req = youtube.videos().insert(
        part="snippet,status,contentDetails",
        body=body,
        media_body=media
    )

    done = False
    retry = 0
    while not done:
        try:
            status, resp = req.next_chunk()
            if status:
                print(f"🟢 {int(status.progress()*100)}% uploaded")
            else:
                done = True
        except HttpError as e:
            if e.resp.status in [500,502,503,504] and retry < max_retries:
                retry += 1
                time.sleep(2**retry)
                print(f"⚠️ Retry {retry}")
            else:
                raise

    print("✅ Upload complete! Video ID:", resp["id"])
    return resp

response = upload_video_to_youtube("shorts_final.mp4", title, description, tags)
print("🎉 Done! https://youtu.be/" + response["id"])